# 60-异步数据库操作与ORM

## 📚 用途说明

**学习目标**：
- 掌握异步数据库操作的核心原理
- 学会使用SQLAlchemy进行异步ORM操作
- 理解数据库连接池和事务管理
- 能够构建高性能的异步数据访问层

**前置要求**：
- 已完成FastAPI和异步编程学习
- 熟练掌握Python异步编程基础
- 了解关系型数据库基本概念
- 理解ORM（对象关系映射）原理

**与LangChain关联**：
- 异步数据库支持LangChain的向量存储
- 高效数据访问提升AI应用性能
- 为LangChain提供持久化对话存储
- 支持大规模AI数据处理和分析

---

## 🔢 知识点覆盖

### 7.5 异步数据库操作与ORM [⭐⭐进阶]
**知识点说明**：异步数据库操作与ORM是构建高性能Web应用的核心技能，对于LangChain的数据存储和检索功能具有重要意义。

**学习要求**：
- 掌握异步数据库操作的核心原理
- 学会使用SQLAlchemy进行异步ORM操作
- 理解数据库连接池和事务管理
- 能够构建高性能的异步数据访问层

**案例要求**：
- 实现完整的异步数据库操作层
- 构建ORM模型和映射关系
- 实现连接池和事务管理
- 验证点：能独立开发异步数据访问系统

In [ ]:
print("🗄️ 异步数据库操作与ORM:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import uuid
import json
import random
import hashlib
from typing import List, Dict, Any, Optional, Union, Type
from dataclasses import dataclass, asdict, field
from enum import Enum
from contextlib import asynccontextmanager

# 数据库相关
import sqlite3
import aiosqlite
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Boolean, Text, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, Session
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession, async_sessionmaker
from sqlalchemy.sql import select, update, delete, insert

# FastAPI相关
from fastapi import FastAPI, HTTPException, Depends, status
from pydantic import BaseModel

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 异步数据库基础
print(f"📝 1. 异步数据库基础:")

# 1.1 数据库连接管理
print(f"\n   🔗 1.1 数据库连接管理:")

@dataclass
class DatabaseConfig:
    """数据库配置"""
    database_url: str
    pool_size: int = 10
    max_overflow: int = 20
    pool_timeout: int = 30
    pool_recycle: int = 3600
    echo: bool = False
    
    def get_async_url(self) -> str:
        """获取异步数据库URL"""
        if self.database_url.startswith("sqlite"):
            return self.database_url.replace("sqlite://", "sqlite+aiosqlite://")
        elif self.database_url.startswith("postgresql"):
            return self.database_url.replace("postgresql://", "postgresql+asyncpg://")
        elif self.database_url.startswith("mysql"):
            return self.database_url.replace("mysql://", "mysql+aiomysql://")
        return self.database_url

class AsyncDatabaseManager:
    """异步数据库管理器"""
    
    def __init__(self, config: DatabaseConfig):
        self.config = config
        self.engine = None
        self.session_factory = None
        self.is_initialized = False
        
        # 统计信息
        self.stats = {
            "total_connections": 0,
            "active_connections": 0,
            "total_queries": 0,
            "failed_queries": 0,
            "connection_errors": 0
        }
    
    async def initialize(self):
        """初始化数据库连接"""
        if self.is_initialized:
            return
        
        try:
            # 创建异步引擎
            self.engine = create_async_engine(
                self.config.get_async_url(),
                pool_size=self.config.pool_size,
                max_overflow=self.config.max_overflow,
                pool_timeout=self.config.pool_timeout,
                pool_recycle=self.config.pool_recycle,
                echo=self.config.echo
            )
            
            # 创建会话工厂
            self.session_factory = async_sessionmaker(
                bind=self.engine,
                class_=AsyncSession,
                expire_on_commit=False
            )
            
            self.is_initialized = True
            
            print(f"      🚀 数据库连接初始化成功")
            print(f"      📊 连接池配置: 大小={self.config.pool_size}, 最大溢出={self.config.max_overflow}")
        
        except Exception as e:
            self.stats["connection_errors"] += 1
            print(f"      ❌ 数据库连接初始化失败: {e}")
            raise
    
    async def close(self):
        """关闭数据库连接"""
        if self.engine:
            await self.engine.dispose()
            self.is_initialized = False
            print(f"      🛑 数据库连接已关闭")
    
    @asynccontextmanager
    async def get_session(self):
        """获取数据库会话"""
        if not self.is_initialized:
            await self.initialize()
        
        async with self.session_factory() as session:
            try:
                self.stats["active_connections"] += 1
                yield session
            except Exception as e:
                await session.rollback()
                self.stats["failed_queries"] += 1
                raise
            finally:
                self.stats["active_connections"] -= 1
    
    async def execute_query(self, query, params=None):
        """执行查询"""
        async with self.get_session() as session:
            try:
                result = await session.execute(query, params or {})
                await session.commit()
                self.stats["total_queries"] += 1
                return result
            except Exception as e:
                self.stats["failed_queries"] += 1
                raise
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        return {
            **self.stats,
            "is_initialized": self.is_initialized,
            "success_rate": (
                (self.stats["total_queries"] - self.stats["failed_queries"]) / 
                max(self.stats["total_queries"], 1) * 100
            )
        }

# 1.2 异步SQLite示例
print(f"\n   🗄️ 1.2 异步SQLite示例:")

class AsyncSQLiteManager:
    """异步SQLite管理器"""
    
    def __init__(self, db_path: str = "async_example.db"):
        self.db_path = db_path
        self.connection = None
        
    async def connect(self):
        """连接数据库"""
        if self.connection is None:
            self.connection = await aiosqlite.connect(self.db_path)
            await self._create_tables()
            print(f"      🗄️ SQLite数据库连接成功: {self.db_path}")
    
    async def disconnect(self):
        """断开连接"""
        if self.connection:
            await self.connection.close()
            self.connection = None
            print(f"      🛑 SQLite数据库连接已关闭")
    
    async def _create_tables(self):
        """创建表"""
        await self.connection.execute("""
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE NOT NULL,
                email TEXT UNIQUE NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                is_active BOOLEAN DEFAULT 1
            )
        """)
        
        await self.connection.execute("""
            CREATE TABLE IF NOT EXISTS posts (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER NOT NULL,
                title TEXT NOT NULL,
                content TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (user_id) REFERENCES users (id)
            )
        """)
        
        await self.connection.commit()
        print(f"      📋 数据表创建完成")
    
    async def insert_user(self, username: str, email: str) -> int:
        """插入用户"""
        cursor = await self.connection.execute(
            "INSERT INTO users (username, email) VALUES (?, ?)",
            (username, email)
        )
        await self.connection.commit()
        return cursor.lastrowid
    
    async def get_user(self, user_id: int) -> Optional[Dict[str, Any]]:
        """获取用户"""
        cursor = await self.connection.execute(
            "SELECT id, username, email, created_at, is_active FROM users WHERE id = ?",
            (user_id,)
        )
        row = await cursor.fetchone()
        
        if row:
            return {
                "id": row[0],
                "username": row[1],
                "email": row[2],
                "created_at": row[3],
                "is_active": bool(row[4])
            }
        return None
    
    async def get_users(self, limit: int = 10) -> List[Dict[str, Any]]:
        """获取用户列表"""
        cursor = await self.connection.execute(
            "SELECT id, username, email, created_at, is_active FROM users LIMIT ?",
            (limit,)
        )
        rows = await cursor.fetchall()
        
        return [
            {
                "id": row[0],
                "username": row[1],
                "email": row[2],
                "created_at": row[3],
                "is_active": bool(row[4])
            }
            for row in rows
        ]
    
    async def update_user(self, user_id: int, **kwargs) -> bool:
        """更新用户"""
        if not kwargs:
            return False
        
        set_clause = ", ".join([f"{k} = ?" for k in kwargs.keys()])
        values = list(kwargs.values()) + [user_id]
        
        cursor = await self.connection.execute(
            f"UPDATE users SET {set_clause} WHERE id = ?",
            values
        )
        await self.connection.commit()
        
        return cursor.rowcount > 0
    
    async def delete_user(self, user_id: int) -> bool:
        """删除用户"""
        cursor = await self.connection.execute(
            "DELETE FROM users WHERE id = ?",
            (user_id,)
        )
        await self.connection.commit()
        
        return cursor.rowcount > 0

print(f"   ✅ 异步数据库基础完成")
print(f"      - DatabaseConfig: 数据库配置")
print(f"      - AsyncDatabaseManager: 异步数据库管理器")
print(f"      - AsyncSQLiteManager: 异步SQLite管理器")

print(f"\n✅ 异步数据库基础完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步数据库操作的核心原理")
print(f"   ✓ 学会使用SQLAlchemy进行异步ORM操作")
print(f"   ✓ 理解数据库连接池和事务管理")
print(f"   ✓ 能够构建高性能的异步数据访问层")

### SQLAlchemy异步ORM [⭐⭐进阶]
**知识点说明**：SQLAlchemy异步ORM是现代Python Web应用的数据访问核心技术，对于LangChain的数据持久化和检索功能具有重要价值。

**学习要求**：
- 掌握SQLAlchemy异步模型定义
- 学会异步查询和操作方法
- 理解关系映射和级联操作
- 能够构建完整的ORM数据层

**案例要求**：
- 实现用户和文章的ORM模型
- 构建异步CRUD操作接口
- 实现复杂查询和关联操作
- 验证点：能独立开发ORM数据访问系统

In [ ]:
print("\n🏗️ SQLAlchemy异步ORM:")
print("=" * 50)

# 2. SQLAlchemy异步ORM
print(f"📝 2. SQLAlchemy异步ORM:")

# 2.1 基础模型定义
print(f"\n   📋 2.1 基础模型定义:")

# 创建基础模型类
Base = declarative_base()

class TimestampMixin:
    """时间戳混入类"""
    created_at = Column(DateTime, default=datetime.datetime.utcnow, nullable=False)
    updated_at = Column(DateTime, default=datetime.datetime.utcnow, onupdate=datetime.datetime.utcnow, nullable=False)

class User(Base, TimestampMixin):
    """用户模型"""
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String(50), unique=True, index=True, nullable=False)
    email = Column(String(100), unique=True, index=True, nullable=False)
    hashed_password = Column(String(255), nullable=False)
    full_name = Column(String(100))
    is_active = Column(Boolean, default=True)
    is_superuser = Column(Boolean, default=False)
    
    # 关系
    posts = relationship("Post", back_populates="author", cascade="all, delete-orphan")
    comments = relationship("Comment", back_populates="author", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<User(id={self.id}, username='{self.username}')>"

class Post(Base, TimestampMixin):
    """文章模型"""
    __tablename__ = "posts"
    
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200), nullable=False, index=True)
    content = Column(Text)
    summary = Column(String(500))
    published = Column(Boolean, default=False)
    view_count = Column(Integer, default=0)
    
    # 外键
    author_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    
    # 关系
    author = relationship("User", back_populates="posts")
    comments = relationship("Comment", back_populates="post", cascade="all, delete-orphan")
    tags = relationship("Tag", secondary="post_tags", back_populates="posts")
    
    def __repr__(self):
        return f"<Post(id={self.id}, title='{self.title[:50]}...')>"

class Comment(Base, TimestampMixin):
    """评论模型"""
    __tablename__ = "comments"
    
    id = Column(Integer, primary_key=True, index=True)
    content = Column(Text, nullable=False)
    is_approved = Column(Boolean, default=False)
    
    # 外键
    author_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    post_id = Column(Integer, ForeignKey("posts.id"), nullable=False)
    
    # 关系
    author = relationship("User", back_populates="comments")
    post = relationship("Post", back_populates="comments")
    
    def __repr__(self):
        return f"<Comment(id={self.id}, post_id={self.post_id})>"

class Tag(Base):
    """标签模型"""
    __tablename__ = "tags"
    
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String(50), unique=True, nullable=False, index=True)
    description = Column(String(200))
    
    # 关系
    posts = relationship("Post", secondary="post_tags", back_populates="tags")
    
    def __repr__(self):
        return f"<Tag(id={self.id}, name='{self.name}')>"

# 多对多关联表
from sqlalchemy import Table
post_tags = Table(
    'post_tags',
    Base.metadata,
    Column('post_id', Integer, ForeignKey('posts.id'), primary_key=True),
    Column('tag_id', Integer, ForeignKey('tags.id'), primary_key=True)
)

# 2.2 异步CRUD操作
print(f"\n   🔧 2.2 异步CRUD操作:")

class AsyncCRUD:
    """异步CRUD操作基类"""
    
    def __init__(self, model: Type[Base], db_manager: AsyncDatabaseManager):
        self.model = model
        self.db_manager = db_manager
    
    async def create(self, obj_data: Dict[str, Any]) -> Base:
        """创建对象"""
        async with self.db_manager.get_session() as session:
            db_obj = self.model(**obj_data)
            session.add(db_obj)
            await session.commit()
            await session.refresh(db_obj)
            return db_obj
    
    async def get(self, obj_id: int) -> Optional[Base]:
        """获取单个对象"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(self.model).where(self.model.id == obj_id)
            )
            return result.scalar_one_or_none()
    
    async def get_multi(self, skip: int = 0, limit: int = 100) -> List[Base]:
        """获取多个对象"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(self.model).offset(skip).limit(limit)
            )
            return result.scalars().all()
    
    async def update(self, obj_id: int, obj_data: Dict[str, Any]) -> Optional[Base]:
        """更新对象"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(self.model).where(self.model.id == obj_id)
            )
            db_obj = result.scalar_one_or_none()
            
            if db_obj:
                for field, value in obj_data.items():
                    setattr(db_obj, field, value)
                
                await session.commit()
                await session.refresh(db_obj)
            
            return db_obj
    
    async def delete(self, obj_id: int) -> bool:
        """删除对象"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(self.model).where(self.model.id == obj_id)
            )
            db_obj = result.scalar_one_or_none()
            
            if db_obj:
                await session.delete(db_obj)
                await session.commit()
                return True
            
            return False
    
    async def count(self) -> int:
        """统计对象数量"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(func.count()).select_from(self.model)
            )
            return result.scalar()

# 2.3 用户CRUD操作
print(f"\n   👤 2.3 用户CRUD操作:")

from sqlalchemy import func

class UserCRUD(AsyncCRUD):
    """用户CRUD操作"""
    
    def __init__(self, db_manager: AsyncDatabaseManager):
        super().__init__(User, db_manager)
    
    async def get_by_username(self, username: str) -> Optional[User]:
        """根据用户名获取用户"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(User).where(User.username == username)
            )
            return result.scalar_one_or_none()
    
    async def get_by_email(self, email: str) -> Optional[User]:
        """根据邮箱获取用户"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(User).where(User.email == email)
            )
            return result.scalar_one_or_none()
    
    async def get_active_users(self) -> List[User]:
        """获取活跃用户"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(User).where(User.is_active == True)
            )
            return result.scalars().all()
    
    async def update_last_login(self, user_id: int):
        """更新最后登录时间"""
        async with self.db_manager.get_session() as session:
            await session.execute(
                update(User).where(User.id == user_id).values(
                    updated_at=datetime.datetime.utcnow()
                )
            )
            await session.commit()
    
    async def get_user_stats(self) -> Dict[str, Any]:
        """获取用户统计"""
        async with self.db_manager.get_session() as session:
            # 总用户数
            total_result = await session.execute(
                select(func.count(User.id))
            )
            total_users = total_result.scalar()
            
            # 活跃用户数
            active_result = await session.execute(
                select(func.count(User.id)).where(User.is_active == True)
            )
            active_users = active_result.scalar()
            
            # 超级用户数
            super_result = await session.execute(
                select(func.count(User.id)).where(User.is_superuser == True)
            )
            super_users = super_result.scalar()
            
            return {
                "total_users": total_users,
                "active_users": active_users,
                "super_users": super_users,
                "inactive_users": total_users - active_users
            }

# 2.4 文章CRUD操作
print(f"\n   📝 2.4 文章CRUD操作:")

class PostCRUD(AsyncCRUD):
    """文章CRUD操作"""
    
    def __init__(self, db_manager: AsyncDatabaseManager):
        super().__init__(Post, db_manager)
    
    async def get_by_author(self, author_id: int, skip: int = 0, limit: int = 10) -> List[Post]:
        """根据作者获取文章"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(Post).where(Post.author_id == author_id)
                .offset(skip).limit(limit)
            )
            return result.scalars().all()
    
    async def get_published_posts(self, skip: int = 0, limit: int = 10) -> List[Post]:
        """获取已发布文章"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(Post).where(Post.published == True)
                .order_by(Post.created_at.desc())
                .offset(skip).limit(limit)
            )
            return result.scalars().all()
    
    async def search_posts(self, keyword: str, skip: int = 0, limit: int = 10) -> List[Post]:
        """搜索文章"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(Post).where(
                    Post.title.contains(keyword) | Post.content.contains(keyword)
                ).offset(skip).limit(limit)
            )
            return result.scalars().all()
    
    async def increment_view_count(self, post_id: int):
        """增加浏览次数"""
        async with self.db_manager.get_session() as session:
            await session.execute(
                update(Post).where(Post.id == post_id).values(
                    view_count=Post.view_count + 1
                )
            )
            await session.commit()
    
    async def get_popular_posts(self, limit: int = 10) -> List[Post]:
        """获取热门文章"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(Post).where(Post.published == True)
                .order_by(Post.view_count.desc())
                .limit(limit)
            )
            return result.scalars().all()
    
    async def get_post_with_author(self, post_id: int) -> Optional[Dict[str, Any]]:
        """获取文章及其作者信息"""
        async with self.db_manager.get_session() as session:
            result = await session.execute(
                select(Post, User).join(User, Post.author_id == User.id)
                .where(Post.id == post_id)
            )
            row = result.first()
            
            if row:
                post, author = row
                return {
                    "post": post,
                    "author": {
                        "id": author.id,
                        "username": author.username,
                        "full_name": author.full_name
                    }
                }
            
            return None

print(f"   ✅ SQLAlchemy异步ORM完成")
print(f"      - Base: 基础模型类")
print(f"      - User: 用户模型")
print(f"      - Post: 文章模型")
print(f"      - Comment: 评论模型")
print(f"      - Tag: 标签模型")
print(f"      - AsyncCRUD: 异步CRUD基类")
print(f"      - UserCRUD: 用户CRUD操作")
print(f"      - PostCRUD: 文章CRUD操作")

print(f"\n✅ SQLAlchemy异步ORM完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握SQLAlchemy异步模型定义")
print(f"   ✓ 学会异步查询和操作方法")
print(f"   ✓ 理解关系映射和级联操作")
print(f"   ✓ 能够构建完整的ORM数据层")

### 高级数据库操作 [⭐⭐进阶]
**知识点说明**：高级数据库操作展示了复杂的数据处理能力，为LangChain的大规模数据管理和分析提供技术支持。

**学习要求**：
- 掌握复杂查询和聚合操作
- 学会事务管理和批量操作
- 理解数据库性能优化技巧
- 能够构建高效的数据处理管道

**案例要求**：
- 实现复杂的数据分析查询
- 构建批量数据处理系统
- 开发数据库性能监控
- 验证点：能独立优化数据库性能

In [ ]:
print("\n🚀 高级数据库操作:")
print("=" * 50)

# 3. 高级数据库操作
print(f"📝 3. 高级数据库操作:")

# 3.1 事务管理
print(f"\n   🔄 3.1 事务管理:")

class AsyncTransactionManager:
    """异步事务管理器"""
    
    def __init__(self, db_manager: AsyncDatabaseManager):
        self.db_manager = db_manager
        self.active_transactions = {}
    
    @asynccontextmanager
    async def transaction(self, session: AsyncSession = None):
        """事务上下文管理器"""
        if session:
            # 使用提供的会话
            transaction = await session.begin()
            try:
                yield session
                await transaction.commit()
            except Exception:
                await transaction.rollback()
                raise
        else:
            # 创建新会话
            async with self.db_manager.get_session() as session:
                transaction = await session.begin()
                try:
                    yield session
                    await transaction.commit()
                except Exception:
                    await transaction.rollback()
                    raise
    
    async def execute_in_transaction(self, operations: List[callable]):
        """在事务中执行多个操作"""
        async with self.transaction() as session:
            results = []
            for operation in operations:
                if asyncio.iscoroutinefunction(operation):
                    result = await operation(session)
                else:
                    result = operation(session)
                results.append(result)
            return results

# 3.2 批量操作
print(f"\n   📦 3.2 批量操作:")

class AsyncBatchOperations:
    """异步批量操作"""
    
    def __init__(self, db_manager: AsyncDatabaseManager):
        self.db_manager = db_manager
    
    async def bulk_insert(self, model: Type[Base], data_list: List[Dict[str, Any]], 
                          batch_size: int = 1000) -> int:
        """批量插入"""
        total_inserted = 0
        
        async with self.db_manager.get_session() as session:
            for i in range(0, len(data_list), batch_size):
                batch = data_list[i:i + batch_size]
                objects = [model(**data) for data in batch]
                
                session.add_all(objects)
                await session.commit()
                
                total_inserted += len(batch)
                print(f"      📦 批量插入: {len(batch)} 条记录 (总计: {total_inserted})")
        
        return total_inserted
    
    async def bulk_update(self, model: Type[Base], updates: List[Dict[str, Any]], 
                          batch_size: int = 1000) -> int:
        """批量更新"""
        total_updated = 0
        
        async with self.db_manager.get_session() as session:
            for i in range(0, len(updates), batch_size):
                batch = updates[i:i + batch_size]
                
                for update_data in batch:
                    obj_id = update_data.pop('id')
                    await session.execute(
                        update(model).where(model.id == obj_id).values(**update_data)
                    )
                
                await session.commit()
                total_updated += len(batch)
                print(f"      📦 批量更新: {len(batch)} 条记录 (总计: {total_updated})")
        
        return total_updated
    
    async def bulk_delete(self, model: Type[Base], ids: List[int], 
                          batch_size: int = 1000) -> int:
        """批量删除"""
        total_deleted = 0
        
        async with self.db_manager.get_session() as session:
            for i in range(0, len(ids), batch_size):
                batch_ids = ids[i:i + batch_size]
                
                await session.execute(
                    delete(model).where(model.id.in_(batch_ids))
                )
                await session.commit()
                
                total_deleted += len(batch_ids)
                print(f"      📦 批量删除: {len(batch_ids)} 条记录 (总计: {total_deleted})")
        
        return total_deleted

# 3.3 复杂查询
print(f"\n   🔍 3.3 复杂查询:")

class AsyncComplexQueries:
    """异步复杂查询"""
    
    def __init__(self, db_manager: AsyncDatabaseManager):
        self.db_manager = db_manager
    
    async def get_user_activity_summary(self, user_id: int) -> Dict[str, Any]:
        """获取用户活动摘要"""
        async with self.db_manager.get_session() as session:
            # 用户基本信息
            user_result = await session.execute(
                select(User).where(User.id == user_id)
            )
            user = user_result.scalar_one_or_none()
            
            if not user:
                return {}
            
            # 发表文章统计
            posts_result = await session.execute(
                select(
                    func.count(Post.id).label('total_posts'),
                    func.sum(Post.view_count).label('total_views'),
                    func.count(func.nullif(Post.published, False)).label('published_posts')
                ).where(Post.author_id == user_id)
            )
            posts_stats = posts_result.first()
            
            # 评论统计
            comments_result = await session.execute(
                select(
                    func.count(Comment.id).label('total_comments'),
                    func.count(func.nullif(Comment.is_approved, False)).label('approved_comments')
                ).where(Comment.author_id == user_id)
            )
            comments_stats = comments_result.first()
            
            return {
                "user": {
                    "id": user.id,
                    "username": user.username,
                    "full_name": user.full_name,
                    "created_at": user.created_at
                },
                "posts": {
                    "total": posts_stats.total_posts or 0,
                    "published": posts_stats.published_posts or 0,
                    "total_views": posts_stats.total_views or 0
                },
                "comments": {
                    "total": comments_stats.total_comments or 0,
                    "approved": comments_stats.approved_comments or 0
                }
            }
    
    async def get_popular_tags(self, limit: int = 10) -> List[Dict[str, Any]]:
        """获取热门标签"""
        async with self.db_manager.get_session() as session:
            result = await session.execute("""
                SELECT 
                    t.id,
                    t.name,
                    t.description,
                    COUNT(pt.post_id) as post_count
                FROM tags t
                LEFT JOIN post_tags pt ON t.id = pt.tag_id
                LEFT JOIN posts p ON pt.post_id = p.id AND p.published = 1
                GROUP BY t.id, t.name, t.description
                ORDER BY post_count DESC
                LIMIT ?
            """, (limit,))
            
            rows = result.fetchall()
            
            return [
                {
                    "id": row[0],
                    "name": row[1],
                    "description": row[2],
                    "post_count": row[3]
                }
                for row in rows
            ]
    
    async def get_recent_activity(self, hours: int = 24) -> Dict[str, List[Dict[str, Any]]]:
        """获取最近活动"""
        since = datetime.datetime.utcnow() - datetime.timedelta(hours=hours)
        
        async with self.db_manager.get_session() as session:
            # 最近文章
            posts_result = await session.execute("""
                SELECT 
                    p.id,
                    p.title,
                    p.created_at,
                    u.username as author_username
                FROM posts p
                JOIN users u ON p.author_id = u.id
                WHERE p.created_at >= ?
                ORDER BY p.created_at DESC
            """, (since,))
            
            recent_posts = [
                {
                    "id": row[0],
                    "title": row[1],
                    "created_at": row[2],
                    "author_username": row[3]
                }
                for row in posts_result.fetchall()
            ]
            
            # 最近评论
            comments_result = await session.execute("""
                SELECT 
                    c.id,
                    c.content,
                    c.created_at,
                    u.username as author_username,
                    p.title as post_title
                FROM comments c
                JOIN users u ON c.author_id = u.id
                JOIN posts p ON c.post_id = p.id
                WHERE c.created_at >= ?
                ORDER BY c.created_at DESC
            """, (since,))
            
            recent_comments = [
                {
                    "id": row[0],
                    "content": row[1][:100] + "..." if len(row[1]) > 100 else row[1],
                    "created_at": row[2],
                    "author_username": row[3],
                    "post_title": row[4]
                }
                for row in comments_result.fetchall()
            ]
            
            return {
                "posts": recent_posts,
                "comments": recent_comments
            }

# 3.4 数据库性能监控
print(f"\n   📊 3.4 数据库性能监控:")

class DatabasePerformanceMonitor:
    """数据库性能监控"""
    
    def __init__(self, db_manager: AsyncDatabaseManager):
        self.db_manager = db_manager
        self.query_stats = {}
        self.slow_queries = []
        self.slow_query_threshold = 1.0  # 秒
    
    async def monitor_query(self, query_name: str, query_func: callable, *args, **kwargs):
        """监控查询性能"""
        start_time = time.time()
        
        try:
            if asyncio.iscoroutinefunction(query_func):
                result = await query_func(*args, **kwargs)
            else:
                result = query_func(*args, **kwargs)
            
            execution_time = time.time() - start_time
            
            # 记录统计
            if query_name not in self.query_stats:
                self.query_stats[query_name] = {
                    "count": 0,
                    "total_time": 0.0,
                    "avg_time": 0.0,
                    "max_time": 0.0,
                    "min_time": float('inf')
                }
            
            stats = self.query_stats[query_name]
            stats["count"] += 1
            stats["total_time"] += execution_time
            stats["avg_time"] = stats["total_time"] / stats["count"]
            stats["max_time"] = max(stats["max_time"], execution_time)
            stats["min_time"] = min(stats["min_time"], execution_time)
            
            # 记录慢查询
            if execution_time > self.slow_query_threshold:
                self.slow_queries.append({
                    "query_name": query_name,
                    "execution_time": execution_time,
                    "timestamp": datetime.datetime.utcnow(),
                    "args": str(args)[:100],
                    "kwargs": str(kwargs)[:100]
                })
                
                # 限制慢查询记录数量
                if len(self.slow_queries) > 100:
                    self.slow_queries = self.slow_queries[-100:]
            
            return result
        
        except Exception as e:
            execution_time = time.time() - start_time
            print(f"      ❌ 查询失败: {query_name} (耗时: {execution_time:.3f}s) - {e}")
            raise
    
    def get_performance_report(self) -> Dict[str, Any]:
        """获取性能报告"""
        # 按平均执行时间排序
        sorted_queries = sorted(
            self.query_stats.items(), 
            key=lambda x: x[1]["avg_time"], 
            reverse=True
        )
        
        return {
            "query_stats": dict(sorted_queries[:10]),  # 前10个最慢查询
            "slow_queries_count": len(self.slow_queries),
            "total_queries": sum(stats["count"] for stats in self.query_stats.values()),
            "recent_slow_queries": self.slow_queries[-5:]  # 最近5个慢查询
        }
    
    def clear_stats(self):
        """清除统计信息"""
        self.query_stats.clear()
        self.slow_queries.clear()

# 创建数据库管理器实例
db_config = DatabaseConfig(
    database_url="sqlite:///./async_blog.db",
    pool_size=5,
    echo=False
)

db_manager = AsyncDatabaseManager(db_config)
transaction_manager = AsyncTransactionManager(db_manager)
batch_operations = AsyncBatchOperations(db_manager)
complex_queries = AsyncComplexQueries(db_manager)
performance_monitor = DatabasePerformanceMonitor(db_manager)

# 创建CRUD实例
user_crud = UserCRUD(db_manager)
post_crud = PostCRUD(db_manager)

print(f"   ✅ 高级数据库操作完成")
print(f"      - AsyncTransactionManager: 事务管理")
print(f"      - AsyncBatchOperations: 批量操作")
print(f"      - AsyncComplexQueries: 复杂查询")
print(f"      - DatabasePerformanceMonitor: 性能监控")

print(f"\n✅ 高级数据库操作完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握复杂查询和聚合操作")
print(f"   ✓ 学会事务管理和批量操作")
print(f"   ✓ 理解数据库性能优化技巧")
print(f"   ✓ 能够构建高效的数据处理管道")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**7.5 异步数据库操作与ORM [⭐⭐进阶]**
- ✅ 掌握异步数据库操作的核心原理
- ✅ 学会使用SQLAlchemy进行异步ORM操作
- ✅ 理解数据库连接池和事务管理
- ✅ 能够构建高性能的异步数据访问层

#### 🎯 核心技能掌握

**异步数据库基础**
- ✅ 异步数据库连接管理
- ✅ 连接池配置和优化
- ✅ 异步SQLite操作
- ✅ 数据库统计和监控

**SQLAlchemy异步ORM**
- ✅ 异步模型定义和关系映射
- ✅ 异步CRUD操作实现
- ✅ 复杂查询和关联操作
- ✅ 用户和文章管理系统

**高级数据库操作**
- ✅ 事务管理和回滚机制
- ✅ 批量操作和性能优化
- ✅ 复杂查询和数据分析
- ✅ 性能监控和慢查询检测

#### 🚀 实践应用能力

**数据访问层设计**
- ✅ 异步数据访问层架构
- ✅ ORM模型设计和优化
- ✅ 数据库连接和会话管理
- ✅ 错误处理和异常管理

**性能优化**
- ✅ 查询性能优化技巧
- ✅ 批量操作和事务优化
- ✅ 连接池配置和管理
- ✅ 数据库监控和调优

#### 📊 与LangChain的关联

**数据持久化**
- ✅ 支持LangChain向量数据库存储
- ✅ 对话历史和用户数据管理
- ✅ 大规模AI数据处理能力
- ✅ 实时数据同步和一致性

**系统集成**
- ✅ 异步数据库与FastAPI集成
- ✅ 数据访问层与业务层分离
- ✅ 缓存和数据库协同工作
- ✅ 微服务数据访问设计

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **异步数据库核心技术** - 理解异步数据库操作原理，掌握连接池管理
2. **SQLAlchemy异步ORM** - 构建完整的ORM数据层，实现复杂查询操作
3. **高级数据处理** - 掌握事务管理、批量操作和性能优化技巧
4. **数据库监控和优化** - 实现性能监控、慢查询检测和系统调优

这些技能为后续的Web安全、微服务架构等高级主题奠定了坚实基础，也为LangChain的大规模数据管理和AI应用开发提供了重要的技术支撑。